In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data


In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# HELPER

In [5]:
# INIT WEIGHTS 
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [6]:
# INIT BIAS
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [7]:
# 2DCONVOLUTION
def conv2d(x,W):
    # x is the input tensor [batch, height of img, width of img,channels]
    # W is the kernel [filter height, filter width, Channels IN, Channels OUT]
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1],padding ='SAME' ) #zero padding = SAME

In [8]:
# POOLING
def max_pool_2by2(x): #takes the max value in the 2x2 matrix
    # x is the input tensor [batch, height of img, width of img,channels]
    #pooling only along height and width
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME') 
    #ksize-> size of the window for each dimension of the input tensor
    #strides->stride of the sliding window

In [9]:
# CONVOLUTIONAL LAYER
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

In [10]:
# NORMAL/ FULLY CONNECTED LAYER
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [11]:
# PLACEHOLDERS
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [12]:
# LAYERS

In [13]:
# reshape the image layer (input) back into 28x28 as it is flattened out 
x_image = tf.reshape(x,[-1,28,28,1])

In [14]:
# convolutional layers  
convo_1 = convolutional_layer(x_image,shape=[6,6,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [15]:
convo_2 = convolutional_layer(convo_1_pooling, shape = [5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [16]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1, 7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))
# 1024 is the no. of neurons

In [17]:
# Dropout (to prevent overfitting)

In [18]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob = hold_prob)

In [19]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [20]:
# LOSS FUNCTION
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_true, logits = y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [21]:
# OPTIMIZER (ADAM)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [22]:
init = tf.global_variables_initializer()

In [ ]:
# RUN THE SESSION
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5}) #hold_prob:probability of holding the neuron(50%)
        
        if i%100 == 0: #every 100 steps
            
            print("ON STEP: ()".format(i))
            print("ACCURACY: ")
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            print(sess.run(acc, feed_dict={x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0}))
            print('\n')